In [1]:
import pandas as pd 
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
#from geopy import distance

In [2]:
df = pd.read_csv('Baseball_Merged.csv')


In [3]:
# RateLimiter must be used for Nominatim
geocoder = RateLimiter(Nominatim(user_agent='ba').geocode, min_delay_seconds=1)
venues = df['venue_name'].unique()

location_dict = {}

for i in venues: 
    location_dict[i] = geocoder(i)


RateLimiter caught an error, retrying (0/2 tries). Called with (*('Wrigley Field',), **{}).
Traceback (most recent call last):
  File "C:\Users\T450s\Anaconda3\lib\site-packages\urllib3\connectionpool.py", line 421, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "C:\Users\T450s\Anaconda3\lib\site-packages\urllib3\connectionpool.py", line 416, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Users\T450s\Anaconda3\lib\http\client.py", line 1344, in getresponse
    response.begin()
  File "C:\Users\T450s\Anaconda3\lib\http\client.py", line 306, in begin
    version, status, reason = self._read_status()
  File "C:\Users\T450s\Anaconda3\lib\http\client.py", line 267, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "C:\Users\T450s\Anaconda3\lib\socket.py", line 589, in readinto
    return self._sock.recv_into(b)
  File "C:\Users\T450s\Anaconda3\lib\site-packages\urllib3\contrib\pyopenssl.py

In [4]:
# there are erroneous pairings
error_pairs = {}
for venue in venues:
    try:
        if location_dict[venue].raw['type'] != 'stadium':
            error_pairs[venue] = location_dict[venue].raw['type']
    except:
        error_pairs[venue] = 'None'

# When looking at online, found good string for geopy
rename_dict = {'Miller Park':'American Family Field',
'O.co Coliseum': 'RingCentral Coliseum',
'U.S. Cellular Field' : 'Guaranteed Rate Field',
'AT&T Park' : 'Oracle Park',
'Oakland Coliseum' : 'RingCentral Coliseum',
'Fort Bragg Field' : 'Fort Bragg Field baseball',
'Williamsport Little League Classic' : 'Williamsport, Pennsylvania',
'Angel Stadium': 'Angel Stadium of Anaheim',
'BB&T Ballpark': 'Williamsport, Pennsylvania'
}

In [5]:
# Get locations to each venue
for k,v in rename_dict.items():
    location_dict[v] = geocoder(v)

for key in [k for k,v in rename_dict.items()]:
    if key in location_dict:
        del location_dict[key]

RateLimiter caught an error, retrying (0/2 tries). Called with (*('American Family Field',), **{}).
Traceback (most recent call last):
  File "C:\Users\T450s\Anaconda3\lib\site-packages\urllib3\connectionpool.py", line 421, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "C:\Users\T450s\Anaconda3\lib\site-packages\urllib3\connectionpool.py", line 416, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Users\T450s\Anaconda3\lib\http\client.py", line 1344, in getresponse
    response.begin()
  File "C:\Users\T450s\Anaconda3\lib\http\client.py", line 306, in begin
    version, status, reason = self._read_status()
  File "C:\Users\T450s\Anaconda3\lib\http\client.py", line 267, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "C:\Users\T450s\Anaconda3\lib\socket.py", line 589, in readinto
    return self._sock.recv_into(b)
  File "C:\Users\T450s\Anaconda3\lib\site-packages\urllib3\contrib\pyop

In [80]:
# we need all date-venue_name pairs for the API 
subdf = df[['date','venue_name']].drop_duplicates()

# we require a new col where new venue names are used
to_replace = [k for k,v in rename_dict.items()]

new_venue = []

for i in subdf['venue_name']:
    if i in to_replace:
        new_venue.append(rename_dict[i])
    else:
        new_venue.append(i)

subdf['new_venue'] = new_venue

# map location based on new venue
subdf['location'] = [location_dict[i] for i in subdf['new_venue']]
subdf['latitude'] = [i.latitude for i in subdf['location']]
subdf['longitude'] = [i.longitude for i in subdf['location']]

In [87]:
subdf.head()

,date,venue_name,new_venue,location,latitude,longitude
0,2015-04-05,Wrigley Field,Wrigley Field,"(Wrigley Field, 1060, West Addison Street, Wri...",41.948122,-87.656351
314,2015-04-06,Safeco Field,Safeco Field,"(T-Mobile Park, 3rd Avenue South, Internationa...",47.591414,-122.332023
567,2015-04-06,Marlins Park,Marlins Park,"(Marlins Park, Northwest 15th Avenue, Miami, M...",25.778247,-80.219805
810,2015-04-06,Tropicana Field,Tropicana Field,"(Tropicana Field, Pinellas Trail, Methodist To...",27.768056,-82.653276
1105,2015-04-06,Citizens Bank Park,Citizens Bank Park,"(Citizens Bank Park, Pattison Avenue, South Ph...",39.905886,-75.165411


In [135]:
subdf.to_csv('date_n_location.csv')

## Get weather data with API 

I will use the Visualcrossing API which is free until 1000 requests/day 

In [214]:
import requests as re 
import json
from datetime import datetime

In [240]:
subdf= pd.read_csv('date_n_location.csv', index_col = 'Unnamed: 0')
subdf['date'] = subdf.apply(lambda x : datetime.strptime(x['date'], "%Y-%m-%d").date(),axis= 1)
subdf = subdf[subdf['date'] > datetime.strptime('2017-04-01','%Y-%m-%d').date()]

In [229]:
def keygen(latitude,longitude,hours,start_date,end_date,json_or_csv,api_key):
    locations = '?locations='+ str(latitude) + ',' + str(longitude)
    aggregateHours = 'aggregateHours=' + str(hours)
    startDateTime = 'startDateTime=' + str(start_date) + 'T00:00:00' 
    endDateTime = 'endDateTime=' + str(end_date) + 'T00:00:00'
    unitGroup = 'unitGroup='+'uk'
    contentType = 'contentType='+json_or_csv
    dayStartTime = 'dayStartTime='+'0:0:00'
    dayEndTime = 'dayEndTime='+'0:0:00'
    key = 'key=' + str(api_key)
    
    
    res = '&'.join([locations,aggregateHours,startDateTime,endDateTime,unitGroup,contentType,dayStartTime,dayEndTime,key])
    request_link = BaseURL+hist_forecast + res
    return(request_link)

In [242]:
to_drop = ['Estadio de Beisbol Monterrey', 'Fort Bragg Field baseball', 'Hiram Bithorn Stadium', 'Williamsport, Pennsylvania']
subdf = subdf[~subdf['new_venue'].isin(to_drop)]

In [243]:
start_df = subdf.groupby(['longitude', 'latitude']).agg({'date':'min'}).rename(columns = {'date':'start_date'}).reset_index()
end_df = subdf.groupby(['longitude', 'latitude']).agg({'date':'max'}).rename(columns = {'date':'end_date'}).reset_index()
api_df = pd.merge(start_df,end_df)
links = api_df.apply(lambda x: keygen(x['latitude'],x['longitude'],hours,x['start_date'],x['end_date'],json_or_csv,api_key) ,axis = 1)

In [234]:
api_df

,longitude,latitude,start_date,end_date
0,-122.390267,37.778612,2017-04-10,2018-09-30
1,-122.332023,47.591414,2017-04-10,2018-09-30
2,-122.199371,37.751675,2017-04-03,2018-09-23
3,-118.240048,34.074552,2017-04-03,2018-10-01
4,-117.881743,33.800260,2017-04-07,2018-09-30
5,-117.156877,32.707188,2017-04-07,2018-09-30
6,-112.066693,33.445486,2017-04-02,2018-09-26
7,-104.992929,39.756031,2017-04-07,2018-09-30
8,-97.083476,32.751390,2017-04-03,2018-09-23
9,-95.355234,29.757232,2017-04-03,2018-09-23


In [249]:
# TODO: sign-up with pay as you go option here https://www.visualcrossing.com/weather-data-editions

In [244]:
request_link = links[0]

In [246]:
myjson = re.get(request_link)

In [247]:
weather_json = json.loads(myjson.text)

In [248]:
weather_json

{'errorCode': 999,
 'executionTime': -1,
 'sessionId': '',
 'message': 'You have exceeded the maximum result row count for your account. Requested 538, Maximum 100. '}

In [187]:
weather_json['locations']['41.9481225,-87.6563513145702']['values']

KeyError: 'locations'

In [ ]:
# TODO: write json parsing part

----------------------------------------

#### Below codes are not used 
----------------------------------------

In [77]:
## NOT USED 

## this is a code to get back city names from geolocations 

# from geopy.geocoders import Nominatim
# geolocator = Nominatim(user_agent="geoapiExercises")

# interim_dict = {}
# city_dict = {}

# for k,v in location_dict.items(): 
#     interim_dict[k]=((location_dict[k].latitude,location_dict[k].longitude))
    
# for k,v in interim_dict.items(): 
#     city_dict[k] = geolocator.reverse(interim_dict[k]).raw['address'].get('city','')
    
# city_dict

{'Wrigley Field': 'Chicago',
 'Safeco Field': 'Seattle',
 'Marlins Park': 'Miami',
 'Tropicana Field': 'St. Petersburg',
 'Citizens Bank Park': 'Philadelphia',
 'Kauffman Stadium': 'Kansas City',
 'Minute Maid Park': 'Houston',
 'Comerica Park': 'Detroit',
 'Nationals Park': 'Washington',
 'Great American Ball Park': 'Cincinnati',
 'Dodger Stadium': 'Los Angeles',
 'Chase Field': 'Phoenix',
 'Yankee Stadium': 'New York',
 'Petco Park': 'San Diego',
 'Coors Field': 'Denver',
 'Progressive Field': 'Cleveland',
 'Globe Life Park in Arlington': 'Arlington',
 'Angel Stadium of Anaheim': 'Anaheim',
 'Turner Field': 'Atlanta',
 'Oriole Park at Camden Yards': 'Baltimore',
 'PNC Park': 'Pittsburgh',
 'Target Field': 'Minneapolis',
 'Busch Stadium': 'City of Saint Louis',
 'Citi Field': 'New York',
 'Rogers Centre': 'Old Toronto',
 'Fenway Park': 'Boston',
 'Guaranteed Rate Field': 'Chicago',
 'SunTrust Park': 'Atlanta',
 'Hiram Bithorn Stadium': 'San Juan',
 'Estadio de Beisbol Monterrey': 'Mon

In [227]:
## This is not used 

# # doing one sample request from the API 

# BaseURL = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/'
# hist_forecast = 'history/'

# latitude = 41.9481225
# longitude = -87.6563513145702
# hours = 24
# start_date = '2019-06-13'
# end_date = '2019-06-13'
# json_or_csv = 'json'
# api_key = 'LB75LJ6SXR4P5SP6U5HA666V6'

# # specifying API parameters 
# locations = '?locations='+ str(latitude) + ',' + str(longitude)
# aggregateHours = 'aggregateHours=' + str(hours)
# startDateTime = 'startDateTime=' + str(start_date) + 'T00:00:00' 
# endDateTime = 'endDateTime=' + str(end_date) + 'T00:00:00'
# unitGroup = 'unitGroup='+'uk'
# contentType = 'contentType='+json_or_csv
# dayStartTime = 'dayStartTime='+'0:0:00'
# dayEndTime = 'dayEndTime='+'0:0:00'
# key = 'key=' + str(api_key)

# # create request link 


# res = '&'.join([locations,aggregateHours,startDateTime,endDateTime,unitGroup,contentType,dayStartTime,dayEndTime,key])

# request_link = BaseURL+hist_forecast + res